# Introduccion a Transformers

In [ ]:
# Uncomment and run if on Colab or Kaggle
!git clone https://github.com/0xpantera/UFM-AI-22.git
%cd UFM-AI-22
from install import *
install_requirements()

Cloning into 'UFM-AI-22'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 41 (delta 16), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (41/41), done.
/content/UFM-AI-22
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


En 2017, los investigadores de Google publicaron un artículo que proponía una arquitectura de red neuronal novedosa para el modelado de secuencias.<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Apodada el Transformer, esta arquitectura superó a las Recurrent Neural Networks (RNN) en tareas de traducción automática, tanto en términos de calidad de traducción como de costo de entrenamiento.

Paralelamente, un método eficaz de transfer learning llamado ULMFiT demostró que entrenar Long Short Term Memory (LSTM) en un corpus muy grande y diverso podría producir clasificadores de texto de última generación con pocos datos etiquetados.<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)

Estos avances fueron los catalizadores de dos de los transformers más conocidos de la actualidad: el Generative Pretrained Transformer (GPT)<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) y Bidirectional Encoder Representations from Transformers (BERT)<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4). Al combinar la arquitectura del transformer con el aprendizaje no supervisado, estos modelos eliminaron la necesidad para entrenar arquitecturas específicas para tareas desde cero y rompió casi todos los puntos de referencia en NLP por un margen significativo. Desde el lanzamiento de GPT y BERT, ha surgido un zoológico de modelos de transformadores:


Para entender qué es lo novedoso de los transformers, primero debemos explicar:

- El framework encoder-decoder
- Mecanismos de attention
- Transfer Learning

En este notebook, veremos los conceptos básicos que subyacen a la omnipresencia de los transformers, haremos un recorrido por algunas de las tareas en las que se destacan y concluiremos con una mirada al ecosistema de herramientas y librerias de Hugging Face.



<a name="cite_note-1"></a>1. [^](#cite_ref-1) A. Vaswani et al., [“Attention Is All You Need”](https://arxiv.org/abs/1706.03762), (2017).

<a name="cite_note-2"></a>2. [^](#cite_ref-2)J. Howard and S. Ruder, [“Universal Language Model Fine-Tuning for Text Classification”](https://arxiv.org/abs/1801.06146), (2018).

<a name="cite_note-3"></a>3. [^](#cite_ref-3)A. Radford et al., [“Improving Language Understanding by Generative Pre-Training”](https://openai.com/blog/language-unsupervised), (2018).

<a name="cite_note-4"></a>4. [^](#cite_ref-4)J. Devlin et al., [“BERT: Pre-Training of Deep Bidirectional Transformers for Language Understanding”](https://arxiv.org/abs/1810.04805), (2018).


## El Framework Encoder-Decoder

Antes de los transformers, las arquitecturas recurrentes como las LSTM eran lo último en NLP. Estas arquitecturas contienen un feedback loop en las conexiones de red que permite que la información se propague de un paso a otro, lo que las hace ideales para modelar datos secuenciales como texto. Como se ilustra en el lado izquierdo de la imagen abajo, un RNN recibe algun input (que podría ser una palabra o un carácter), la alimenta a través de la red y genera un vector llamado _hidden state_. Al mismo tiempo, el modelo se retroalimenta con cierta información a través del feedback loop, que luego puede usar en el siguiente paso. Esto se puede ver más claramente si "desenrollamos" el ciclo como se muestra en el lado derecho de la imagen: la RNN pasa información sobre su estado en cada paso a la siguiente operación en la secuencia. Esto permite que una RNN le de seguimiento a la información de los pasos anteriores y la use para sus predicciones de output.


Estas arquitecturas fueron (y continúan siendo) ampliamente utilizadas para tareas de NLP, procesamiento de voz y series temporales. Pueden encontrar una exposición muy buena de sus capacidades en la publicación del blog de Andrej Karpathy, [“The Unreasonable Effectiveness of Recurrent Neural Networks”](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

Un área en la que las RNN jugaron un papel importante fue en el desarrollo de sistemas de traducción automática, donde el objetivo es mapear una secuencia de palabras en un idioma a otro. Este tipo de tarea generalmente se aborda con una arquitectura de encoder-decoder o de secuencia a secuencia,[5] que es adecuada para situaciones en las que el input y le output son secuencias de longitud arbitraria. El trabajo del encoder es codificar la información de la secuencia de input a una representación numérica que se llama el _last hidden state_. Este estado luego se pasa al decoder, que genera la secuencia de output.

En general, los componentes encoder y decoder pueden ser cualquier tipo de arquitectura de red neuronal que pueda modelar secuencias. Esto se ilustra para un par de RNN en la imagen abajo, donde la oración en inglés "Transformers are great!" se codifica como un vector de hidden state que luego se decodifica para producir la traducción al alemán “Transformer sind grossartig!” Las palabras de input se alimentan secuencialmente a través del codificador y las palabras de output se generan una a la vez, de arriba a abajo.


Aunque es elegante y simple, una debilidad de esta arquitectura es que el _last hidden state_ del encoder crea un _information bottleneck_: tiene que representar el significado de toda la secuencia de input porque esto es todo a lo que tiene acceso el decoder cuando genera el output. Esto es especialmente dificil para secuencias largas, donde la información al comienzo de la secuencia puede perderse en el proceso de comprimir todo en una sola representación fija.

Hay una forma de salir de este bottleneck al permitir que el decoder tenga acceso a todos los _hidden states_ del encoder. El mecanismo general para esto se llama _attention_,[6] y es un componente clave en muchas arquitecturas de redes neuronales modernas. Entender cómo se desarrolló _attention_ para los RNN nos va a ayudar a entender uno de los principales componentes básicos de la arquitectura de Transformer.

5 I. Sutskever, O. Vinyals, and Q.V. Le, [“Sequence to Sequence Learning with Neural Networks”](https://arxiv.org/abs/1409.3215), (2014).

6 D. Bahdanau, K. Cho, and Y. Bengio, [“Neural Machine Translation by Jointly Learning to Align and Trans‐ late”](https://arxiv.org/abs/1409.0473), (2014).

# Mecanismos de Attention

La idea principal detrás de _attention_ es que en lugar de producir un solo _hidden state_ para la secuencia de input, el encoder genera un _hidden state_ en cada paso al que puede acceder el decoder. Sin embargo, usar todos los estados al mismo tiempo crearía una entrada enorme para el decoder, por lo que se necesita algún mecanismo para priorizar qué estados usar. Aquí es donde entra attention: permite que el decoder asigne una cantidad diferente de peso, o "_attention_", a cada uno de los estados del encoder en cada timestep de decodificación. Este proceso se ilustra en la imagen, donde se muestra el papel de la atención para predecir el tercer token en la secuencia de salida.

IMAGEN AQUI

Al enfocarse en qué tokens del input son más relevantes en cada timestep, estos modelos basados en _attention_ pueden aprender alineaciones no triviales entre las palabras en una traducción generada y las de una oración de origen. Por ejemplo, la imagen abajo visualiza los pesos de attention para un modelo de traducción del inglés al francés, donde cada píxel denota un peso. La imagen muestra cómo el decoder es capaz de alinear correctamente las palabras “zone” y “Area”, que están ordenadas de manera diferente en los dos idiomas.

IMAGEN AQUI

Aunque _attention_ permitió la producción de traducciones mucho mejores, todavía existía una gran deficiencia con el uso de modelos recurrentes para el encoder y el decoder: los cálculos son inherentemente secuenciales y no se pueden paralelizar a lo largo de la secuencia de input.

Con el transformer, se introdujo un nuevo paradigma de modelado: dejar por completo la recurrencia y, en cambio, confiar completamente en una forma especial de _attention_ llamada _self-attention_. Mas adelante vamos a ver que es _self-attention_ con más detalle, pero la idea básica es permitir que _attention_ opere en todos los estados en la misma capa de la red neuronal. Esto se muestra en la imagen abajo, donde tanto el encoder como el decoder tienen sus propios mecanismos de _self-attention_, cuyos outputs se alimentan a las redes neuronales _feed-forward_ (FF NN). Esta arquitectura se puede entrenar mucho más rápido que los modelos recurrentes y lidero el camino para muchos de los avances recientes en NLP.

IMAGEN AQUI

En el paper original de Transformer, el modelo de traducción se entrenó desde cero en un gran corpus de pares de oraciones en varios idiomas. Sin embargo, en muchas aplicaciones prácticas de NLP no tenemos acceso a grandes cantidades de datos de texto etiquetados para entrenar nuestros modelos. Faltaba una pieza final para que comenzara la revolución de los transformers: el _transfer learning_.

# Transfer Learning en NLP

Hoy en día, es una práctica común en computer vision utilizar transfer learning para entrenar una red neuronal convolucional como ResNet en una tarea y luego adaptarla o ajustarla en una nueva tarea. Esto permite que la red haga uso del conocimiento aprendido en la tarea original. Arquitectónicamente, esto implica dividir el modelo en su cuerpo y su cabeza, donde la cabeza es una red de tareas específicas. Durante el entrenamiento, los pesos (parametros aprenidods por la red) del cuerpo aprenden características generales del dominio de origen y estos pesos se utilizan para inicializar un nuevo modelo para la nueva tarea. En comparación con el aprendizaje supervisado tradicional, este enfoque generalmente produce modelos de alta calidad que pueden entrenarse de forma mas eficiente en una variedad de tareas downstream y con mucho menos data etiquetada. Una comparacion de los dos enfoques abajo:

IMAGEN AQUI

En computer vision, los modelos primero se entrenan en conjuntos de datos a gran escala como [ImageNet](https://image-net.org), que contienen millones de imágenes. Este proceso se denomina _pretraining_ y su objetivo principal es enseñar a los modelos las _features_ básicas de las imágenes, como los bordes o los colores. Estos modelos previamente entrenados se pueden ajustar en una tarea posterior, como la clasificación de especies de flores con una cantidad relativamente pequeña de ejemplos etiquetados (generalmente unos pocos cientos por clase). Los modelos ajustados suelen lograr una mayor precisión que los modelos supervisados entrenados desde cero con la misma cantidad de datos etiquetados.

Aunque el _transfer learning_ se convirtió en el enfoque estándar en computer vision, durante muchos años no estuvo claro cuál era el proceso de _pretraining_ análogo para NLP. Como resultado, las aplicaciones de NLP generalmente requerían grandes cantidades de datos etiquetados para lograr un alto rendimiento. Incluso entonces, ese desempeño no se comparó con lo que se logró en el dominio de CV.

En 2017 y 2018, varios grupos de investigación propusieron nuevos enfoques que finalmente hicieron que el _transfer learning_ funcionara para NLP. Comenzó con una perspectiva de los investigadores de OpenAI que obtuvieron un rendimiento sólido en una tarea de clasificación de sentimientos mediante el uso de _features_ extraídas de un entrenamiento previo no supervisado.[8] A esto le siguió ULMFiT, que introdujo un framework general para adaptar modelos LSTM _pretrained_ para varias tareas.[9]
Como pueden ver abajo, ULMFiT implica tres pasos principales:

_Pretraining_
- El objetivo de entrenamiento inicial es bastante simple: predecir la siguiente palabra en base a las palabras anteriores. Esta tarea se conoce como _language modeling_. La elegancia de este enfoque es que no se requieren datos etiquetados, y uno puede hacer uso de la gran cantidad de texto disponible de fuentes como Wikipedia.[10]

_Domain adaptation_
- Una vez que el _language model_ se entrena previamente en un corpus a gran escala, el siguiente paso es adaptarlo al corpus del dominio (por ejemplo, de Wikipedia al corpus de reviews de películas de IMDb). Esta etapa aún utiliza _language modeling_, pero ahora el modelo tiene que predecir la siguiente palabra en el corpus de destino.

_Fine-tuning_
- En este paso, el _language model_ se ajusta con una capa de clasificación para la tarea de destino (por ejemplo, clasificar el sentimiento de los reviews de películas).

Al introducir un framework viable de _pretraining_ y _transfer learning_ en NLP, ULMFiT proporcionó la pieza que faltaba para que los transformers despegaran. En 2018, se lanzaron dos transformers que combinaron _self-attention_ con _transfer learning_:

GPT
- Utiliza solo la parte del decoder de la arquitectura Transformer y el mismo enfoque de _language modeling_ que ULMFiT. GPT se entrenó previamente en BookCorpus,[11] que consta de 7000 libros inéditos de una variedad de géneros, incluidos Aventura, Fantasía y Romance.

BERT
Utiliza la parte del encoder de la arquitectura Transformer y una forma especial de _language modeling_ llamado _masked language modeling_. El objetivo de este es predecir palabras enmascaradas (_masked_) de forma aleatoria en un texto. Por ejemplo, dada una oración como "I looked at my [MASK] and saw that [MASK] was late.". el modelo necesita predecir los candidatos más probables para las palabras enmascaradas que se indican con [MASK]. BERT fue _pretrained_ en BookCorpus y Wikipedia en inglés.

GPT y BERT establecieron un nuevo _state of the art_ (SOTA) en una variedad de _benchmarks_ de NLP y marcaron el comienzo de la era de los transformers.

Sin embargo, con diferentes labs de investigación lanzando sus modelos en frameworks incompatibles (PyTorch o TensorFlow), no siempre fue fácil para los que trabajaban en NLP traducir estos modelos a sus propias aplicaciones. Con el lanzamiento de [Transformers](https://github.com/huggingface/transformers), se construyó progresivamente una API unificada para más de 50 arquitecturas. Esta libreria catalizó la explosión de research sobre transformers y rápidamente llegó a los profesionales de NLP, lo que facilita la integración de estos modelos en muchas aplicaciones de la vida real.

8 A. Radford, R. Jozefowicz, and I. Sutskever, [“Learning to Generate Reviews and Discovering Sentiment”](https://arxiv.org/abs/1704.01444), (2017).

11 Y. Zhu et al., [“Aligning Books and Movies: Towards Story-Like Visual Explanations by Watching Movies and Reading Books”](https://arxiv.org/abs/1506.06724), (2015).

# Un tour de aplicaciones de Transformer

Cada tarea de NLP comienza con un fragmento de texto, como este comentario inventado de un cliente sobre un pedido en línea:

In [1]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
    from your online store in Germany. Unfortunately, when I opened the package,
    I discovered to my horror that I had been sent an action figure of Megatron
    instead! As a lifelong enemy of the Decepticons, I hope you can understand my
    dilemma. To resolve the issue, I demand an exchange of Megatron for the
    Optimus Prime figure I ordered. Enclosed are copies of my records concerning
    this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

Dependiendo de la aplicación, el texto con el que estamos trabajando podría ser un contrato legal, una descripción del producto o algo completamente diferente. En el caso de los comentarios de clientes, probablemente nos gustaría saber si los comentarios son positivos o negativos. Esta tarea se llama _sentiment analysis_ y es parte del tema más amplio de clasificación de texto que veremos despues. Por ahora, veamos que se necesita para extraer el sentimiento de nuestro texto usando Transformers.

## Clasificacion de texto

Como veremos en despues, Transformers tiene una API en capas que nos permite interactuar con la libreria en varios niveles de abstracción. En este notebook, comenzaremos con las _pipelines_, que abstraen todos los pasos necesarios para convertir texto crudo en un conjunto de predicciones a partir de un modelo _fine-tuned_.

En Transformers, instanciamos un _pipeline_ llamando la función `pipeline()` y le damos el nombre de la tarea que nos interesa:

In [2]:
from transformers import pipeline 

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


La primera vez que ejecuten este código, verán que aparecen algunas barras de progreso porque el _pipeline_ descarga automáticamente los pesos del modelo de [Hugging Face Hub](https://huggingface.co/models). La segunda vez que creen una instancia del _pipeline_, la libreria notará que ya descargaron los pesos y utilizará la versión en cache. Por default, el _pipeline_ de clasificación de texto utiliza un modelo diseñado para _sentiment analysis_, pero también es compatible con la clasificación multiclase y multilabel.

Ahora que tenemos nuestra pipeline, generemos algunas predicciones. Cada pipeline toma una string de texto (o una lista de strings) como input y devuelve una lista de predicciones. Cada predicción es un diccionario de Python, por lo que podemos usar Pandas para verlas bien como un DataFrame:

In [3]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


En este caso, el modelo está muy seguro de que el texto tiene un sentimiento negativo, lo cual tiene sentido porque estamos lidiando con una queja de un cliente enojado. Tomen en cuenta que para las tareas de _sentiment analysis_, el pipeline solo devuelve una de las etiquetas POSITIVE o NEGATIVE, ya que la otra se puede inferir calculando un score.

# Named Entity Recognition

Predecir el sentimiento de los comentarios de los clientes es un buen primer paso, pero casi siempre uno quiere saber si los comentarios se refieren a un artículo o servicio en particular. En NLP, los objetos del mundo real como productos, lugares y personas se denominan _named entities_, y extraerlos del texto se denomina _named entity recognition_ (NER). Podemos aplicar NER cargando el pipeline correspondiente y alimentándola con el review de nuestro cliente:

In [4]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,94,101
3,MISC,0.556571,Mega,216,220
4,PER,0.590255,##tron,220,224
5,ORG,0.669693,Decept,265,271
6,MISC,0.498350,##icons,271,276
7,MISC,0.775362,Megatron,366,374
8,MISC,0.987854,Optimus Prime,387,400
9,PER,0.812096,Bumblebee,526,535


Podemos ver que el pipeline detectó todas las entidades y también asignó una categoría como ORG (organization), LOC (location) o PER (person) a cada una de ellas. Aquí usamos el argumento `aggregation_strategy` para agrupar las palabras de acuerdo con las predicciones del modelo. Por ejemplo, la entidad “Optimus Prime” está compuesta por dos palabras, pero se le asigna una sola categoría: MISC (miscellaneous). Los puntajes nos dicen que tan confiado estaba el modelo sobre las entidades que identificó. Podemos ver que tenía menos confianza en los "Decepticons" y la primera vez que aparece "Megatron", los cuales no logró agrupar como una sola entidad.

Los hashes (#) que aparecen el la columna `word` son producidos por el _tokenizer_ del modelo que parte palabras en unidades atomicas llamadas _tokens_. Vamos a ver este tema mas a detalle despues.

Extraer todas las entidades nombradas funciona pero a veces queremos hacer preguntas mas detalladas. Aqui es donde podemos usar _question answering_.

## Question Answering

Al responder preguntas, proporcionamos al modelo un pasaje de texto llamado _context_, junto con una pregunta cuya respuesta nos gustaría extraer. Luego, el modelo devuelve el tramo de texto correspondiente a la respuesta. Veamos qué obtenemos cuando hacemos una pregunta específica sobre los comentarios de nuestros clientes:

In [5]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

,score,start,end,answer
0,0.631291,351,374,an exchange of Megatron


Podemos ver que junto con la respuesta, el pipeline también devolvió enteros en las columnas `start` y `end` que corresponden a los índices de caracteres donde se encontró el intervalo de la respuesta (al igual que con NER). Hay varios tipos de respuesta a preguntas que vamos a ver despues, pero este tipo particular se llama _extractive question answering_ porque la respuesta se extrae directamente del texto.

Con este enfoque, podemos leer y extraer información relevante rápidamente de los comentarios de un cliente. Pero, ¿qué sucede si recibimos una montaña de quejas largas y no tenemos tiempo para leerlas todas?

## Summarization

El objetivo del resumen de texto es tomar un texto largo como input y generar una versión corta con todos los hechos relevantes. Esta es una tarea mucho más complicada que las anteriores ya que requiere que el modelo genere un texto coherente. Podemos instanciar un pipeline de _summarization_ de la siguiente manera:

In [6]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that he had been sent an action figure of Megatron instead.


El resumen es bastante bueno. Aunque se han copiado partes del texto original, el modelo pudo captar la esencia del problema e identificar correctamente que “Bumblebee” (que aparecía al final) era el autor de la denuncia. En este ejemplo, también podemos ver que pasamos algunos _keyword arguments_ como `max_length` y `clean_up_tokenization_spaces` al pipelin. Estos nos dejan ajustar los outpus en runtime.

Pero, ¿qué pasa cuando recibimos comentarios en un idioma que no entendemos? Podemos usar Google Translate, o podemos usar nuestro propio transformer para traducirlo por nosotros.

## Translation

Como _summarization_, la traduccion es una tarea donde el output consiste en generar texto. Probemos usar un pipleine para traducir un texto en ingles a aleman:

In [7]:
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])


Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


El modelo produjo una muy buena traducción que usa correctamente los pronombres formales del alemán, como “Ihrem” y “Sie”. Aquí también mostramos cómo podemos anular el modelo predeterminado en el pipeline para elegir el mejor para su aplicación, y podemos encontrar modelos para miles de pares de idiomas en Hugging Face Hub. Antes ver todo el ecosistema Hugging Face, veamos una última aplicación.

## Text Generation

Supongamos que nos gustaria poder proporcionar respuestas más rápidas a los comentarios de los clientes al tener acceso a una función de autocompletar. Con un modelo de generación de texto, lo hacemos asi:

In [8]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure
    from your online store in Germany. Unfortunately, when I opened the package,
    I discovered to my horror that I had been sent an action figure of Megatron
    instead! As a lifelong enemy of the Decepticons, I hope you can understand my
    dilemma. To resolve the issue, I demand an exchange of Megatron for the
    Optimus Prime figure I ordered. Enclosed are copies of my records concerning
    this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. Please refer to Etsy.com for more information.


Unfortunately, as a resident of Germany, I cannot provide any assistance with my own collection. Fortunately, the Optimus Prime action figures arrived in my country about 10


Probablmente no queremos usar este resultado para calmar a Bumblebee, pero podemos ver la idea general.

Ahora que hemos visto algunas aplicaciones de los transformers, probablemente queremos ver donde ocurre el entrenamiento. Todos los modelos que hemos usado en este notebook están disponibles públicamente y ya están ajustados para la tarea en cuestión. Sin embargo, generalmente vamos a querer ajustar los modelos con nuestros propios datos, y en las proximas clases vamos a aprender cómo hacerlo.

Pero entrenar un modelo es solo una pequeña parte de cualquier proyecto de NLP: poder procesar datos de manera eficiente, compartir resultados con colaboradoer y hacer que nuestro trabajo sea reproducible también son componentes clave. Afortunadamente, Hugging Face Transformers tiene un buen ecosistema de herramientas útiles que respaldan gran parte del workflow moderno de ML.